In [1]:
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
import  os

In [2]:
class Regressor(keras.layers.Layer):

    def __init__(self):
        super(Regressor, self).__init__()

        # here must specify shape instead of tensor !
        # name here is meanless !
        # [dim_in, dim_out]
        self.w = self.add_variable('meanless-name', [13, 1])
        # [dim_out]
        self.b = self.add_variable('meanless-name', [1])

        print(self.w.shape, self.b.shape)
        print(type(self.w), tf.is_tensor(self.w), self.w.name)
        print(type(self.b), tf.is_tensor(self.b), self.b.name)


    def call(self, x):

        x = tf.matmul(x, self.w) + self.b

        return x

In [3]:
# test Linear Regression on boston housing prediction
def main():
    
    tf.random.set_seed(22)
    np.random.seed(22)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    assert tf.__version__.startswith('2.')

    (x_train, y_train), (x_val, y_val) = keras.datasets.boston_housing.load_data()
    
    x_train, x_val = x_train.astype(np.float32), x_val.astype(np.float32)
    # (404, 13) (404,) (102, 13) (102,)
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
    # mis-leading issues:
    # 1. (x_train, y_train) cannot be written as [x_train, y_train]
    db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
    db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(102)

    model = Regressor()
    criteon = keras.losses.MeanSquaredError()
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)

    for epoch in range(200):

        for step, (x, y) in enumerate(db_train):

            with tf.GradientTape() as tape:
                # [b, 1]
                logits = model(x)
                # [b]
                logits = tf.squeeze(logits, axis=1)
                # [b] vs [b]
                loss = criteon(y, logits)

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(epoch, 'loss:', loss.numpy())


        if epoch % 10 == 0:

            for x, y in db_val:
                # [b, 1]
                logits = model(x)
                # [b]
                logits = tf.squeeze(logits, axis=1)
                # [b] vs [b]
                loss = criteon(y, logits)

                print(epoch, 'val loss:', loss.numpy())

In [4]:
if __name__ == '__main__':
    main()

57344/57026 [==============================] - 0s 0us/step


W1009 14:13:33.442121 4624827840 deprecation.py:323] From <ipython-input-2-59a106cfbed4>:9: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


(404, 13) (404,) (102, 13) (102,)
(13, 1) (1,)
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0
0 loss: 39695.023
0 val loss: 36500.55
1 loss: 18709.295
2 loss: 6534.532
3 loss: 1271.5453
4 loss: 51.911457
5 loss: 236.73438
6 loss: 401.17065
7 loss: 293.0813
8 loss: 127.25572
9 loss: 48.90026
10 loss: 42.267822
10 val loss: 124.677475
11 loss: 49.233364
12 loss: 47.35646
13 loss: 42.244392
14 loss: 39.96756
15 loss: 39.822285
16 loss: 39.75222
17 loss: 39.398033
18 loss: 39.11085
19 loss: 38.935925
20 loss: 38.738976
20 val loss: 120.31867
21 loss: 38.4842
22 loss: 38.209473
23 loss: 37.937233
24 loss: 37.66733
25 loss: 37.395893
26 loss: 37.11916
27 loss: 36.833473
28 loss: 36.537872
29 loss: 36.234394
30 loss: 35.925827
30 val loss: 114.44217
31 loss: 35.613995
32 loss: 35.29952
33 loss: 34.98245
34 loss: 34.662853
35 loss: 34.341164
36 loss: 34.01807